In [105]:
import os

from langchain_community.vectorstores import SupabaseVectorStore
from langchain_core.documents import Document
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.chat_models import ChatCohere
from supabase.client import Client, create_client
import dotenv, os

dotenv.load_dotenv()
supabase_url = os.getenv("SUPABASE_URL")
print(supabase_url)
supabase_key = os.getenv("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

user_query = "what is a linked list"

API_KEY_OK = os.getenv('COHERE_API_KEY')

# Create cohere's chat model and embeddings objects
cohere_chat_model = ChatCohere(cohere_api_key=API_KEY_OK)
cohere_embeddings = CohereEmbeddings(cohere_api_key=API_KEY_OK)

https://ejicwtmscywsdzsvxngo.supabase.co


In [123]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("D:\code\shlokathon\intellectquest\documents\Array.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [124]:
vector_store = SupabaseVectorStore.from_documents(
    docs,
    cohere_embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
    chunk_size=500,
)

2024-03-18 02:49:05,249:INFO - HTTP Request: POST https://ejicwtmscywsdzsvxngo.supabase.co/rest/v1/documents "HTTP/1.1 201 Created"


In [39]:
vector_store = SupabaseVectorStore(
    embedding=cohere_embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)

In [111]:
matched_docs = vector_store.similarity_search(user_query)

2024-03-18 02:27:24,469:INFO - HTTP Request: POST https://ejicwtmscywsdzsvxngo.supabase.co/rest/v1/rpc/match_documents?limit=4 "HTTP/1.1 200 OK"


In [117]:
print(matched_docs[0].page_content)


Linked List Module Documentation

Introduction
------------
A linked list is a linear data structure where elements are not stored at contiguous memory locations. Each element is a separate object called a "node" that contains data and a reference (link) to the next node in the sequence.

Concepts and Definitions
------------------------
- **Node**: An individual part of a linked list containing the data and the link to the next node.
- **Head**: The first node in a linked list.
- **Tail**: The last node in a linked list, which points to NULL.

Use Cases
---------
- Implementing stacks, queues, graphs, and other complex data structures.
- Dynamic memory allocation where the size of the data structure can change over time.


In [102]:
from langchain.retrievers import ContextualCompressionRetriever, CohereRagRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.chat_models import ChatCohere
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
import os, dotenv
import cohere

In [118]:
co = cohere.Client(os.getenv('COHERE_API_KEY'))


In [121]:
import re

page_content = matched_docs[0].page_content

# Split the page content into sections based on the pattern
sections = re.split(r'\n-{2,}\n', page_content)

# Extract the title and snippet from each section
result = []
for section in sections:
    lines = section.strip().split('\n')
    title = lines[0].strip()
    snippet = '\n'.join(lines[1:]).strip()
    result.append({"title": title, "snippet": snippet})

print(result)


[{'title': 'Linked List Module Documentation', 'snippet': '================================\n\nIntroduction'}, {'title': 'A linked list is a linear data structure where elements are not stored at contiguous memory locations. Each element is a separate object called a "node" that contains data and a reference (link) to the next node in the sequence.', 'snippet': 'Concepts and Definitions'}, {'title': '- **Node**: An individual part of a linked list containing the data and the link to the next node.', 'snippet': '- **Head**: The first node in a linked list.\n- **Tail**: The last node in a linked list, which points to NULL.\n\nUse Cases'}, {'title': '- Implementing stacks, queues, graphs, and other complex data structures.', 'snippet': '- Dynamic memory allocation where the size of the data structure can change over time.'}]


In [122]:
response = co.chat(
        message=user_query,
        documents = result)
print(response.text)

A linked list is a linear data structure comprising elements known as nodes. Each node contains data and a reference (link) to the next node in the sequence. The first node in the linked list is called the head, while the last node is referred to as the tail, which points to NULL. Linked lists are useful because they allow for dynamic memory allocation, where the size of the data structure can change over time.
